# Advanced Lane Finding Project

The goals / steps of this project are the following:

- Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
- Apply a distortion correction to raw images.
- Use color transforms, gradients, etc., to create a thresholded binary image.
- Apply a perspective transform to rectify binary image ("birds-eye view").
- Detect lane pixels and fit to find the lane boundary.
- Determine the curvature of the lane and vehicle position with respect to center.
- Warp the detected lane boundaries back onto the original image.
- Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Line class 

In [128]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  

## Finding corners

In [217]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

leftLine = Line()
rightLine = Line()

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

## Calibrating camera

In [130]:
# TODO: Write a function that takes an image, object points, and image points
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
def cal_undistort(img, objpoints, imgpoints):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# f.tight_layout()
# ax1.imshow(imgDistort)
# ax1.set_title('Original Image', fontsize=50)
# ax2.imshow(imgUndistort)
# ax2.set_title('Undistorted Image', fontsize=50)
# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Transform perspective

In [131]:
def corners_unwarp(img, nx, ny):
    src = np.float32([[585, 450], [685, 450], [200, 700], [1100, 700]])
    dst = np.float32([[200, 0], [1100, 0], [200, 700], [1100, 700]])
    M = cv2.getPerspectiveTransform(src, dst)
    img_size = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, M

def toNormalView(color_warp, imgUndistort):
    src = np.float32([[585, 450], [685, 450], [200, 700], [1100, 700]])
    dst = np.float32([[200, 0], [1100, 0], [200, 700], [1100, 700]])
    M = cv2.getPerspectiveTransform(dst, src)
    img_size = (imgUndistort.shape[1], imgUndistort.shape[0])
    warped = cv2.warpPerspective(color_warp, M, img_size, flags=cv2.INTER_LINEAR)
    return warped

## Color transform

In [132]:
def binaryImg(img):
    # Convert to HLS color space and separate the S channel
    # Note: img is the undistorted image
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]

    # Grayscale image
    # NOTE: we already saw that standard grayscaling lost color information for the lane lines
    # Explore gradients in other colors spaces / color channels to see what might work better
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    thresh_min = 20
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Threshold color channel
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    # Plotting thresholded images
#     f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
#     ax1.set_title('Stacked thresholds')
#     ax1.imshow(color_binary)

#     ax2.set_title('Combined S channel and gradient thresholds')
#     ax2.imshow(combined_binary, cmap='gray')
    return combined_binary

## Finding lane pixels

In [133]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

## Fitting polynomial

In [143]:
def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return left_fit, right_fit, left_fitx, right_fitx, ploty, out_img

## Fitting poly

In [135]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

## Searching around poly

In [193]:
def search_around_poly(left_fit, right_fit, binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    print(nonzero)
    print(nonzeroy)
    print(nonzerox)
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    print(left_lane_inds)
    print(left_lane_inds)
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result

## Measuring curvature pixels

In [194]:
def measure_curvature_pixels(left_fit, right_fit, ploty):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''  
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

## Measuring real curvature

In [195]:
def measure_curvature_real(left_fit, right_fit, ploty):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''    
    left_fit_cr = left_fit
    right_fit_cr = right_fit
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

In [211]:
def caliAndWarp(image):    
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    imgUndistort = cal_undistort(image, objpoints, imgpoints)
    imgBinary = binaryImg(imgUndistort)
    warpedImg, m = corners_unwarp(imgBinary, 9, 6)
    left_fit, right_fit, left_fitx, right_fitx, ploty, out_img = fit_polynomial(warpedImg)
    return left_fit, right_fit, left_fitx, right_fitx, ploty, out_img, warpedImg, m

In [212]:
def process_image(image):
    left_fit, right_fit, left_fitx, right_fitx, ploty, out_img, warpedImg, m = caliAndWarp(image)
    #print(left_fit)
    #print(right_fit)
    #result = search_around_poly(left_fit, right_fit, warpedImg)
    #plt.imshow(result)

    # Calculate the radius of curvature in pixels for both lane lines
    left_curveradPx, right_curveradPx = measure_curvature_pixels(left_fit, right_fit, ploty)
    print(left_curveradPx, right_curveradPx)

    # Calculate the radius of curvature in meters for both lane lines
    left_curverad, right_curverad = measure_curvature_real(left_fit, right_fit, ploty)
    print(left_curverad, 'm', right_curverad, 'm')
    
    # Calculate offset of the car
    camera_center = (left_fitx[-1] + right_fitx[-1]) / 2
    center_diff = (camera_center - warpedImg.shape[1]/2) * xm_per_pix
    side_pos = 'left'
    if center_diff <= 0:
        side_pod = 'right'
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warpedImg).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    print(pts)
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newWarp = toNormalView(color_warp, imgUndistort)
    result = cv2.addWeighted(imgUndistort, 1, newWarp, 0.3, 0) 
    return result

## Pipeline

In [214]:
# imgDistort1 = cv2.imread('test_images/straight_lines1.jpg')
# imgDistort1 = cv2.cvtColor(imgDistort1, cv2.COLOR_RGB2BGR)
# imgUndistort1 = cal_undistort(imgDistort1, objpoints, imgpoints)
# imgBinary1 = binaryImg(imgUndistort1)
# warpedImg1, m = corners_unwarp(imgBinary1, 9, 6)
# left_fit, right_fit, left_fitx, right_fitx, ploty, out_img = fit_polynomial(warpedImg1)

# result = process_image(imgDistort1)
# plt.imshow(result)
# cv2.destroyAllWindows()
images = glob.glob('test_images/test*.jpg')
for fname in images:
    image = cv2.imread(fname)
    result = process_image(image)
    result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
    cv2.imshow('img', result)
    cv2.waitKey(500)
#cv2.destroyAllWindows()

sad
925.6440716998588 1835.9532781638904
1714.4259293533255 m 2500.3971987214913 m
[[[5.29165472e+02 0.00000000e+00]
  [5.28419686e+02 1.00000000e+00]
  [5.27674981e+02 2.00000000e+00]
  ...
  [1.35061854e+03 2.00000000e+00]
  [1.35112826e+03 1.00000000e+00]
  [1.35163852e+03 0.00000000e+00]]]
sad
951.5885027469249 770.9760456081476
1557.5758124410124 m 1425.0942617190415 m
[[[109.348601     0.        ]
  [110.01474664   1.        ]
  [110.67982549   2.        ]
  ...
  [906.53447519   2.        ]
  [905.74503026   1.        ]
  [904.95421399   0.        ]]]
sad
1609.0221368060477 1978.4337986649043
2790.829665092086 m 2854.3124655938186 m
[[[6.25662101e+02 0.00000000e+00]
  [6.24904179e+02 1.00000000e+00]
  [6.24146945e+02 2.00000000e+00]
  ...
  [1.38694169e+03 2.00000000e+00]
  [1.38753147e+03 1.00000000e+00]
  [1.38812178e+03 0.00000000e+00]]]
sad
1897.1063425987986 1871.976748345768
2368.4998453061644 m 2546.4204555662513 m
[[[4.57499853e+02 0.00000000e+00]
  [4.57082950e+02 1.000

In [ ]:
images = glob.glob('test_images/test*.jpg')
# Step through the list and search for chessboard corners
for fname in images:
    imgDistort = cv2.imread(fname)
    imgDistort = cv2.cvtColor(imgDistort, cv2.COLOR_RGB2BGR)
    imgUndistort = cal_undistort(imgDistort, objpoints, imgpoints)
    weimgBinary = binaryImg(imgUndistort)
    warpedImg, m = corners_unwarp(imgBinary, 9, 6)
    left_fit, right_fit, left_fitx, right_fitx, ploty, out_img = fit_polynomial(warpedImg)
    #plt.imshow(out_img)
    cv2.imshow('img',out_img)
    cv2.waitKey(500)

In [220]:
Input_video = 'project_video.mp4'
Output_video = 'output2_tracked.mp4'
clip1 = VideoFileClip(Input_video).cutout(0, 7)
video_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
video_clip.write_videofile(Output_video, audio=False)


                                                                                                                       
t:  56%|█████████████████████████████████████                             | 707/1260 [26:06<11:39,  1.27s/it, now=None]
                                                                                                                       
t:  41%|██████████████████████████▊                                       | 440/1085 [09:48<14:05,  1.31s/it, now=None]

t:   0%|                                                                            | 0/1085 [00:00<?, ?it/s, now=None]

803.2945341218417 623.4674718673102
1547.502801045861 m 1957.3317198753282 m
[[[ 30.22139385   0.        ]
  [ 31.01448327   1.        ]
  [ 31.80630033   2.        ]
  ...
  [732.45103611   2.        ]
  [731.33436909   1.        ]
  [730.21609493   0.        ]]]
Moviepy - Building video output2_tracked.mp4.
Moviepy - Writing video output2_tracked.mp4





t:   0%|▏                                                                   | 2/1085 [00:01<11:05,  1.63it/s, now=None]

803.2945341218417 623.4674718673102
1547.502801045861 m 1957.3317198753282 m
[[[ 30.22139385   0.        ]
  [ 31.01448327   1.        ]
  [ 31.80630033   2.        ]
  ...
  [732.45103611   2.        ]
  [731.33436909   1.        ]
  [730.21609493   0.        ]]]




t:   0%|▏                                                                   | 3/1085 [00:02<14:29,  1.24it/s, now=None]

...



t: 100%|█████████████████████████████████████████████████████████████████| 1085/1085 [22:48<00:00,  1.23s/it, now=None]

40930.31248200073 7730.762280798182
40739.709569848594 m 7915.888935688943 m
[[[3.86539503e+02 0.00000000e+00]
  [3.86456962e+02 1.00000000e+00]
  [3.86374395e+02 2.00000000e+00]
  ...
  [1.25250369e+03 2.00000000e+00]
  [1.25264119e+03 1.00000000e+00]
  [1.25277883e+03 0.00000000e+00]]]


30486.474841593845 3577.6964597140836
30323.92090431138 m 3748.8375323678792 m
[[[3.75712255e+02 0.00000000e+00]
  [3.75646278e+02 1.00000000e+00]
  [3.75580269e+02 2.00000000e+00]
  ...
  [1.25309384e+03 2.00000000e+00]
  [1.25328026e+03 1.00000000e+00]
  [1.25346696e+03 0.00000000e+00]]]


                                                                                                                       
t:  56%|█████████████████████████████████████                             | 707/1260 [48:56<11:39,  1.27s/it, now=None]
                                                                                                                       
t:  41%|██████████████████████████▊                                       | 440/1085 [32:38<14:05,  1.31s/it, now=None]

Moviepy - Done !
Moviepy - video ready output2_tracked.mp4
